# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [ ]:
!pip install openai
!pip install jupyter_bokeh

In [2]:
import openai

In [3]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

This is added from the original lab

In [6]:
#add
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [7]:
#add
import openai
import os
from dotenv import load_dotenv

load_dotenv()  # Load .env file
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client with the API key
openai.api_key = api_key

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    # Assuming there's more to the function, like sending the request

In [ ]:
#add
#Print openai_api_key to make sure it holds the correct value
print("API Key: ", openai_api_key)

end of the added api key issue

In [9]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [10]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

the response is giving issues. I added another coomand underneith

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [21]:
import time
import openai

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    for attempt in range(1, 6):  # Retry up to 5 times
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
            )
            return response.choices[0].message.content
        except openai.error.RateLimitError as e:
            print(f"Rate limit error, retrying attempt {attempt}.")
            time.sleep(2 ** attempt)  # Exponential backoff
    raise Exception("Failed to get completion after retries due to rate limit errors")

end of the added response

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [25]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

In [ ]:
import openai

# Ensure your API key is set up (this key should be securely managed in a real application)

def get_order_response(user_input):
    prompt = f"""
    You are an OrderBot for a pizza restaurant. Your job is to take orders from customers. 
    A customer says: "{user_input}"
    Your response:
    """
    response = openai.Completion.create(
        engine="gpt-3.5-turbo",
        prompt=prompt,
        temperature=0.5,
        max_tokens=150,
        n=1,
        stop=None,
    )
    return response.choices[0].text.strip()

def order_bot():
    print("Welcome to the Pizza Restaurant!")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("OrderBot: Thank you for visiting! Have a great day!")
            break
        response = get_order_response(user_input)
        print(f"OrderBot: {response}")

# Initiate the bot
order_bot()

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [1]:
import openai

# Assume this API key has been securely set in your environment variable
#openai.api_key = "your-openai-api-key"

# Define a sample pizza menu (this can be expanded or modified)
pizza_menu = {
    "Margherita": "Tomato sauce, mozzarella, and basil",
    "Pepperoni": "Tomato sauce, mozzarella, and pepperoni slices",
    "Hawaiian": "Tomato sauce, mozzarella, ham, and pineapple",
    "Veggie": "Tomato sauce, mozzarella, bell peppers, onions, mushrooms",
}



In [2]:
def format_menu():
    return "\n".join([f"{name}: {description}" for name, description in pizza_menu.items()])


In [3]:

def get_order_response(user_input):
    prompt = f"""
    You are an OrderBot for a pizza restaurant. You take pizza orders based on the available menu.
    
    Here is the menu:
    {format_menu()}

    Customers may also customize their orders with additional toppings.
    Your tasks:
    - Take customer pizza orders, confirm the selected options.
    - Answer questions about the menu.
    - Allow customization with extra toppings.
    
    A customer says: "{user_input}"
    Your response:
    """
    response = openai.Completion.create(
        engine="gpt-3.5-turbo",
        prompt=prompt,
        max_tokens=150,
        temperature=0.5,
        n=1,
        stop=None
    )
    return response.choices[0].text.strip()



In [ ]:
def order_bot():
    print("Welcome to the Pizza Restaurant! Here's our menu:")
    print(format_menu())
    print("To exit, type 'exit', 'quit', or 'bye'.")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("OrderBot: Thank you for visiting! Enjoy your meal!")
            break
        response = get_order_response(user_input)
        print(f"OrderBot: {response}")

# Start the bot
order_bot()

Excercise 2

In [1]:
import openai

# Assume your OpenAI API key is securely set (ensure environmental security in real usage)


# Define a function to get responses from OpenAI's language model
def get_order_response(user_input):
    prompt = f"""
    You are an OrderBot for a pizza restaurant. You take pizza orders based on the available menu.

    Menu:
    - Margherita: Tomato sauce, mozzarella, and basil
    - Pepperoni: Tomato sauce, mozzarella, and pepperoni slices
    - Hawaiian: Tomato sauce, mozzarella, ham, and pineapple
    - Veggie: Tomato sauce, mozzarella, bell peppers, onions, mushrooms

    Customers can customize their orders with additional toppings. 
    Your task is to help customers with menu inquiries, take orders, and suggest popular options.

    A customer says: "{user_input}"
    OrderBot response:
    """
    response = openai.Completion.create(
        engine="gpt-3.5-turbo",
        prompt=prompt,
        max_tokens=150,
        temperature=0.5,
        n=1,
    )
    return response.choices[0].text.strip()


In [ ]:
input("You: ")

In [ ]:

# Function for interactive ordering
'''def order_bot():
    print("Welcome to the Pizza Restaurant! Here's our menu:")
    print("""
    - Margherita: Tomato sauce, mozzarella, and basil
    - Pepperoni: Tomato sauce, mozzarella, and pepperoni slices
    - Hawaiian: Tomato sauce, mozzarella, ham, and pineapple
    - Veggie: Tomato sauce, mozzarella, bell peppers, onions, mushrooms
    """)
    print("To exit, type 'exit', 'quit', or 'bye'.")
    '''
def collect_messages():
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("OrderBot: Thank you for visiting! Enjoy your meal!")
            break
        response = get_order_response(user_input)
        print(f"OrderBot: {response}")

# Start the bot
order_bot()